<div class='bar_title'></div>

*Simulation for Decision Making (S4DM)*

# Assignment 3: Introduction to Simulation with Python (Part 4)

Gunther Gust & Ignacio Ubeda <br>
Chair for Enterprise AI <br>
Data Driven Decisions Group <br>
Center for Artificial Intelligence and Data Science (CAIDAS)


<img src="images/d3.png" style="width:20%; float:left;" />

<img src="images/CAIDASlogo.png" style="width:20%; float:left;" />

# Agenda

* Logging (*101*)
* Condition events (waiting for multiple events)


Credits: The following content is adapted from the official [movie renege example](https://simpy.readthedocs.io/en/latest/examples/movie_renege.html) 

# Movie renege example

### Scenario

- This examples models a movie theater with one ticket counter selling tickets for three different movies. 

- People arrive at random times and try to buy a random number (1–6) of tickets for a random movie. 

- When a movie is sold out, all people waiting to buy a ticket for that movie renege (leave the queue).

In [25]:
import simpy
import random


Let's recall our convention, 

- Resources will be modeled by a python class
- Entities will be modeled by a python class
- Entities arrivals will be modeled by a python function (*simpy process*)

Remember that when possible, we'll use this convention to model our system

### Resource (Theater)

Alongise the simpy environment (`self.env`) and the simpy resource (`self.counter`), we'll create a set of attributes in the `__init__` method to keep track of some relevant information that we would like to analyze later (**this is one (our first) way of logging some information without just printing it**). In particular:

- `self.available`: This creates a dictionary that maps each movie to the number of available tickets.
- `self.sold_out`: This creates a dictionary that maps each movie to a `simpy.Event`. The event will be triggered when the movie is sold out.
- `self.when_sold_out`: This creates a dictionary that maps each movie to the time when it was sold out. Initially, this is `None` for all movies.
- `self.num_renegers`: This creates a dictionary that maps each movie to the number of customers who left without buying a ticket because the movie was sold out (the *renegers*).

**Note 1:** Check the following references if you're new to python data structures. We'll mostly use [lists](https://www.geeksforgeeks.org/python-lists/), [dictionaries](https://www.geeksforgeeks.org/python-dictionary/) and [(pandas) dataframes](https://www.geeksforgeeks.org/python-pandas-dataframe/). Keep in mind that there are more types like sets, tuples, (numpy) arrays, etc. [Here](https://www.geeksforgeeks.org/python-data-structures/) is more info if you want to dive deeper. 

**Note 2:** We are using [python comprehensions](https://www.netguru.com/blog/python-list-comprehension-dictionary) to create the logging attributes. This is a shorthand for a loop such that `x = {movie: tickets for movie in self.movies}` is equivalent to:

```
x = {}
for movie in self.movies:
    x[movie] = tickets
```






In [26]:
class Theater:

    def __init__(self, env, movies, sellout_threshold, tickets):
        '''
        Initialize the theater.

        - env: the simulation environment
        - movies: a list of movie names
        - sellout_threshold: the number of tickets remaining at which a movie is considered sold out
        - tickets: the number of tickets available for each movie
        '''
        
        self.env = env
        self.counter = simpy.Resource(self.env, capacity=1)
        
        self.movies = movies
        self.sellout_threshold = sellout_threshold
        
        #"logging" attributes
        self.available = {movie: tickets for movie in self.movies}
        self.sold_out = {movie: self.env.event() for movie in self.movies}
        self.when_sold_out = {movie: None for movie in self.movies}
        self.num_renegers = {movie: 0 for movie in self.movies}
        
    def sell_tickets(self, movie, num_tickets):
        '''
        Process for selling tickets to a movie.

        - movie: the name of the movie
        - num_tickets: the number of tickets to sell
        '''

        # Buy tickets
        self.available[movie] -= num_tickets
        if self.available[movie] < self.sellout_threshold:
            # Trigger the "sold out" event for the movie
            self.sold_out[movie].succeed()
            self.when_sold_out[movie] = self.env.now
            self.available[movie] = 0
        
        yield self.env.timeout(1)

The `sell_tickets` method is the process supported by the `Theater` resource. This method simulates the process of selling tickets for a movie. The method takes two parameters: `movie` (the name of the movie) and `num_tickets` (the number of tickets to sell).

1. First, we decreases the number of available tickets for the movie: `self.available[movie] -= num_tickets` (equivalently: `self.available[movie] = self.available[movie] - num_tickets`).

2. Then, we check if the number of available tickets for the movie is less than the sellout threshold. If so:
    - `self.sold_out[movie].succeed()` triggers the "sold out" event for the movie.
    - `self.when_sold_out[movie] = self.env.now` records the current simulation time as the time when the movie was sold out.
    - `self.available[movie] = 0` sets the number of available tickets for the movie to 0 (i.e. no more available tickets for that movie)

3. Lastly, this process takes 1 (determinstic) unit of time (`self.env.timeout(1)`)

### Entity (Customer)

We initialize the `Customer` class with the simpy environment (`self.env`) and the `Theater` object. We also store the chosen movie (`self.movie`) and the number of tickets to buy (`self.num_tickets`)

Note that, unlike the previous examples, we now call the `.run()` method within the initialization of the instance. Both options are posible and in some cases we will prefer one over other (more on this in the next lecture!).

In [27]:
class Customer:

    def __init__(self, env, theater):
        '''
        Initialize a customer entity.

        - env: the simulation environment
        - theater: the theater object
        '''

        self.env = env
        self.movie = random.choice(theater.movies)
        self.num_tickets = random.randint(1, 6)

        self.env.process(self.run(theater))
        
    def discuss(self):
        '''
        Customer discusses with the cashier about the movie being sold out.
        '''

        yield self.env.timeout(0.5)

    def run(self, theater):  
        '''
        Entity process for a customer buying tickets to a movie (flow)
        
        - theater: the theater object
        '''

        with theater.counter.request() as req:
            
            # Wait until it's our turn or until the movie is sold out
            result = yield req | theater.sold_out[self.movie]

            # Check if it's our turn or if movie is sold out
            if req not in result:
                theater.num_renegers[self.movie] += 1
                return

            # Check if enough tickets left.
            if theater.available[self.movie] < self.num_tickets:
                # Customer leaves after some discussion
                yield self.env.process(self.discuss())
                return

            yield self.env.process(theater.sell_tickets(self.movie, self.num_tickets))

The `run` method defines the flow of the customer through the theater.

1. First, we request access to the simpy resource (`theater.counter.request()`). Recall that the simpy resource is stored in the `counter` attribute of the `Theater` class.

2. Then `result = yield req | theater.sold_out[movie]` refers to the *Condition* event. Recall that `|` refers to a *logical or* which means that `result` will store the value of the event that succeeds ("happens") first.

3. Since `result` will store the event that suceeds first, `req not in result` means that the `sold_out` suceeds first, therefore we add one to the *reneges* and `return` to end the flow of the entity.

4. If `req` suceeds first, we will continue with the flow of the entity. However we have to check if there are enough tickets first: 
    - If not, the customer also leaves but after some discussion, which takes 0.5 (deterministic) unit of times. The `return` is again for ending the flow of the entity.
    - If yes, then the `sell_tickets` process is yielded.

### Entity Generator

The entity generation process is handled by `customer_arrivals`. You should be already familiar with this structure from the CarWash example (previous assignment).

In [28]:
def customer_arrivals(env, theater):
    '''
    Entity generator for customers arriving at the theater.

    - env: the simulation environment
    - theater: the theater object

    '''
    
    while True:
        yield env.timeout(random.expovariate(1 / 0.5))
        Customer(env, theater)

### Run Simulation

In [29]:
RANDOM_SEED = 42
TICKETS = 50  # Number of tickets per movie
SELLOUT_THRESHOLD = 2  # Fewer tickets than this is a sellout
SIM_TIME = 120  # Simulate until

# Setup and start the simulation
print('Movie renege')
print('Running Simulation...')
random.seed(RANDOM_SEED)
env = simpy.Environment()

#define resources
movies = ['Python Unchained', 'Kill Process', 'Pulp Implementation']
theater = Theater(env, movies, SELLOUT_THRESHOLD, TICKETS)

#define processes
env.process(customer_arrivals(env, theater))

# Execute
env.run(until=SIM_TIME)
print('... Done \n')

# Analysis/results
for movie in movies:
    sellout_time = theater.when_sold_out[movie]
    sellout_time = sellout_time if sellout_time is not None else ''
    num_renegers = theater.num_renegers[movie]
    print(f'Movie: "{movie}"')
    print(f'Sold out {sellout_time:.1f} minutes after ticket counter opening')
    print(f'Number of people leaving due lack of tickets: {num_renegers}')
    print('-' * 30)

Movie renege
Running Simulation...
... Done 

Movie: "Python Unchained"
Sold out 38.0 minutes after ticket counter opening
Number of people leaving due lack of tickets: 52
------------------------------
Movie: "Kill Process"
Sold out 43.0 minutes after ticket counter opening
Number of people leaving due lack of tickets: 39
------------------------------
Movie: "Pulp Implementation"
Sold out 28.0 minutes after ticket counter opening
Number of people leaving due lack of tickets: 53
------------------------------


# Exercise / Tasks

**Recall: Tasks are built on top of each other to progressively build the final solution**

### Task 1:  Logging more *renegers*

Note that the *renegers* we're logging are only the ones that leave the system because the movie was sold out. However we're still missing the ones that discussed with the cashier because the number of tickets they wanted was greater than the number of tickets available.

**Task 1: Modify the `run` method of the `Customer` class to also account for this *renegers*.**

**Note:** For now, we're not interested in differentiating the "type" of *renegers*. Therefore you can use the same `num_renegers` attribute to register all of them.

In [30]:
class Customer:

    def __init__(self, env, theater):
        '''
        Initialize a customer entity.

        - env: the simulation environment
        - theater: the theater object
        '''

        self.env = env
        self.movie = random.choice(theater.movies)
        self.num_tickets = random.randint(1, 6)

        self.env.process(self.run(theater))
        
    def discuss(self):
        '''
        Customer discusses with the cashier about the movie being sold out.
        '''

        yield self.env.timeout(0.5)

    def run(self, theater):  
        '''
        Entity process for a customer buying tickets to a movie (flow)
        
        - theater: the theater object
        '''

        with theater.counter.request() as req:
            
            # Wait until it's our turn or until the movie is sold out
            result = yield req | theater.sold_out[self.movie]

            # Check if it's our turn or if movie is sold out
            if req not in result:
                theater.num_renegers[self.movie] += 1
                return

            # Check if enough tickets left.
            if theater.available[self.movie] < self.num_tickets:
                # Customer leaves after some discussion
                yield self.env.process(self.discuss())
                theater.num_renegers[self.movie] += 1
                return

            yield self.env.process(theater.sell_tickets(self.movie, self.num_tickets))

In [31]:
#Leave this code chunk as is to check your changes

RANDOM_SEED = 42
TICKETS = 50  # Number of tickets per movie
SELLOUT_THRESHOLD = 2  # Fewer tickets than this is a sellout
SIM_TIME = 120  # Simulate until

# Setup and start the simulation
print('Movie renege')
print('Running Simulation...')
random.seed(RANDOM_SEED)
env = simpy.Environment()

#define resources
movies = ['Python Unchained', 'Kill Process', 'Pulp Implementation']
theater = Theater(env, movies, SELLOUT_THRESHOLD, TICKETS)

#define processes
env.process(customer_arrivals(env, theater))

# Execute
env.run(until=SIM_TIME)
print('... Done \n')

# Analysis/results
for movie in movies:
    sellout_time = theater.when_sold_out[movie]
    sellout_time = sellout_time if sellout_time is not None else ''
    num_renegers = theater.num_renegers[movie]
    print(f'Movie: "{movie}"')
    print(f'Sold out {sellout_time:.1f} minutes after ticket counter opening')
    print(f'Number of people leaving due lack of tickets: {num_renegers}')
    print('-' * 30)

Movie renege
Running Simulation...
... Done 

Movie: "Python Unchained"
Sold out 38.0 minutes after ticket counter opening
Number of people leaving due lack of tickets: 74
------------------------------
Movie: "Kill Process"
Sold out 43.0 minutes after ticket counter opening
Number of people leaving due lack of tickets: 72
------------------------------
Movie: "Pulp Implementation"
Sold out 28.0 minutes after ticket counter opening
Number of people leaving due lack of tickets: 88
------------------------------


### Task 2:  Logging lost sales (tickets)

We have now logged the *renegers* correctly but we still don't know how many tickets we are losing due to customers leaving the system.

**Task 2.1: Create a new attribute in the `Theater` class, `lost_tickets`, to record the number of lost tickets due *renegers*.**

**Task 2.2: Modify the `run` method of the `Customer` class to record the lost tickets for each movie.**

In [32]:
class Theater:

    def __init__(self, env, movies, sellout_threshold, tickets):
        '''
        Initialize the theater.

        - env: the simulation environment
        - movies: a list of movie names
        - sellout_threshold: the number of tickets remaining at which a movie is considered sold out
        - tickets: the number of tickets available for each movie
        '''
        
        self.env = env
        self.counter = simpy.Resource(self.env, capacity=1)
        
        self.movies = movies
        self.sellout_threshold = sellout_threshold
        
        #"logging" attributes
        self.available = {movie: tickets for movie in self.movies}
        self.sold_out = {movie: self.env.event() for movie in self.movies}
        self.when_sold_out = {movie: None for movie in self.movies}
        self.num_renegers = {movie: 0 for movie in self.movies}
        
        self.lost_tickets = {movie: 0 for movie in self.movies}
        
    def sell_tickets(self, movie, num_tickets):
        '''
        Process for selling tickets to a movie.

        - movie: the name of the movie
        - num_tickets: the number of tickets to sell
        '''

        # Buy tickets
        self.available[movie] -= num_tickets
        if self.available[movie] < self.sellout_threshold:
            # Trigger the "sold out" event for the movie
            self.sold_out[movie].succeed()
            self.when_sold_out[movie] = self.env.now
            self.available[movie] = 0
        
        yield self.env.timeout(1)

In [33]:
class Customer:

    def __init__(self, env, theater):
        '''
        Initialize a customer entity.

        - env: the simulation environment
        - theater: the theater object
        '''

        self.env = env
        self.movie = random.choice(theater.movies)
        self.num_tickets = random.randint(1, 6)

        self.env.process(self.run(theater))
        
    def discuss(self):
        '''
        Customer discusses with the cashier about the movie being sold out.
        '''

        yield self.env.timeout(0.5)

    def run(self, theater):  
        '''
        Entity process for a customer buying tickets to a movie (flow)
        
        - theater: the theater object
        '''

        with theater.counter.request() as req:
            
            # Wait until it's our turn or until the movie is sold out
            result = yield req | theater.sold_out[self.movie]

            # Check if it's our turn or if movie is sold out
            if req not in result:
                theater.num_renegers[self.movie] += 1
                theater.lost_tickets[self.movie] += self.num_tickets
                return

            # Check if enough tickets left.
            if theater.available[self.movie] < self.num_tickets:
                # Customer leaves after some discussion
                yield self.env.process(self.discuss())
                theater.num_renegers[self.movie] += 1
                theater.lost_tickets[self.movie] += self.num_tickets
                return

            yield self.env.process(theater.sell_tickets(self.movie, self.num_tickets))

In [34]:
#Leave this code chunk as is to check your changes

RANDOM_SEED = 42
TICKETS = 50  # Number of tickets per movie
SELLOUT_THRESHOLD = 2  # Fewer tickets than this is a sellout
SIM_TIME = 120  # Simulate until

# Setup and start the simulation
print('Movie renege')
print('Running Simulation...')
random.seed(RANDOM_SEED)
env = simpy.Environment()

#define resources
movies = ['Python Unchained', 'Kill Process', 'Pulp Implementation']
theater = Theater(env, movies, SELLOUT_THRESHOLD, TICKETS)

#define processes
env.process(customer_arrivals(env, theater))

# Execute
env.run(until=SIM_TIME)
print('... Done \n')

# Analysis/results
for movie in movies:
    num_renegers = theater.num_renegers[movie]
    lost_tickets = theater.lost_tickets[movie]
    print(f'Movie: "{movie}"')
    print(f'Number of people leaving due lack of tickets: {num_renegers}')
    print(f'Number of tickets lost due due lack of tickets: {lost_tickets}')
    print('-' * 30)        

Movie renege
Running Simulation...
... Done 

Movie: "Python Unchained"
Number of people leaving due lack of tickets: 74
Number of tickets lost due due lack of tickets: 257
------------------------------
Movie: "Kill Process"
Number of people leaving due lack of tickets: 72
Number of tickets lost due due lack of tickets: 241
------------------------------
Movie: "Pulp Implementation"
Number of people leaving due lack of tickets: 88
Number of tickets lost due due lack of tickets: 320
------------------------------


Answer the following questions:

- How many tickets are lost for each movie?

**Answer in this markdown chunk:**

- *_Python unchained_ 257*
- *_Kill Process_* 241
- *_Pulp Implementation_* 320

### Task 3: Impatient *renegers*

What about if customers can also leave the queue for impatience as in the bank renege example? In this task you'll implement this but now we're also interested in logging this information.

For modeling this, we'll assume the **patience follows a uniform distribution between 5 and 10 units of time**.

**Task 3.1: Create two new attributes in the `Theater` class, `num_renegers_impatience` and `lost_tickets_impatience`, to record the number of *renegers* and lost tickets due impatience, respectively.**

**Task 3.2: Modify the `run` method of the `Customer` to implement customers leaving the queue due impatience. Use the attributes of Task 3.1. to log each time this happens.**

**Hint:** Note that you need to add an *or* condition in the `result` variable an differentiate between *renegers* that leaves because tickets sold out and those who leave due impatience.

In [74]:
class Theater:

    def __init__(self, env, movies, sellout_threshold, tickets):
        '''
        Initialize the theater.

        - env: the simulation environment
        - movies: a list of movie names
        - sellout_threshold: the number of tickets remaining at which a movie is considered sold out
        - tickets: the number of tickets available for each movie
        '''
        
        self.env = env
        self.counter = simpy.Resource(self.env, capacity=1)
        
        self.movies = movies
        self.sellout_threshold = sellout_threshold
        
        #"logging" attributes
        self.available = {movie: tickets for movie in self.movies}
        self.sold_out = {movie: self.env.event() for movie in self.movies}
        self.when_sold_out = {movie: None for movie in self.movies}


        self.num_renegers = {movie: 0 for movie in self.movies}
        self.lost_tickets = {movie: 0 for movie in self.movies}
        self.num_renegers_impatience = {movie: 0 for movie in self.movies}
        self.lost_tickets_impatience = {movie: 0 for movie in self.movies}


        
    def sell_tickets(self, movie, num_tickets):
        '''
        Process for selling tickets to a movie.

        - movie: the name of the movie
        - num_tickets: the number of tickets to sell
        '''

        # Buy tickets
        self.available[movie] -= num_tickets
        if self.available[movie] < self.sellout_threshold:
            # Trigger the "sold out" event for the movie
            self.sold_out[movie].succeed()
            self.when_sold_out[movie] = self.env.now
            self.available[movie] = 0
        
        yield self.env.timeout(1)

In [75]:
class Customer:

    def __init__(self, env, theater):
        '''
        Initialize a customer entity.

        - env: the simulation environment
        - theater: the theater object
        '''

        self.env = env
        self.movie = random.choice(theater.movies)
        self.num_tickets = random.randint(1, 6)
        self.patience = random.uniform(5, 10)

        self.env.process(self.run(theater))
        
    def discuss(self):
        '''
        Customer discusses with the cashier about the movie being sold out.
        '''

        yield self.env.timeout(0.5)

    def run(self, theater):  
        '''
        Entity process for a customer buying tickets to a movie (flow)
        
        - theater: the theater object
        '''   
        with theater.counter.request() as req:
            
            # Wait until it's our turn or until the movie is sold out or lost patience
            results = yield req | theater.sold_out[self.movie] | self.env.timeout(self.patience)
             
            # Check if it's our turn or if movie is sold out
            if req not in results:
                    if theater.sold_out[self.movie] in results:
                        theater.num_renegers[self.movie] += 1
                        theater.lost_tickets[self.movie] += self.num_tickets
                    else:
                        theater.num_renegers_impatience[self.movie] += 1
                        theater.lost_tickets_impatience[self.movie] += self.num_tickets
                    return

            # Check if enough tickets left.
            if theater.available[self.movie] < self.num_tickets:
                # Customer leaves after some discussion
                yield self.env.process(self.discuss())
                theater.num_renegers[self.movie] += 1
                theater.lost_tickets[self.movie] += self.num_tickets
                return
            
            yield self.env.process(theater.sell_tickets(self.movie, self.num_tickets))


In [76]:
#Leave this code chunk as is to check your changes

RANDOM_SEED = 42
TICKETS = 50  # Number of tickets per movie
SELLOUT_THRESHOLD = 2  # Fewer tickets than this is a sellout
SIM_TIME = 120  # Simulate until

# Setup and start the simulation
print('Movie renege')
print('Running Simulation...')
random.seed(RANDOM_SEED)
env = simpy.Environment()

#define resources
movies = ['Python Unchained', 'Kill Process', 'Pulp Implementation']
theater = Theater(env, movies, SELLOUT_THRESHOLD, TICKETS)

#define processes
env.process(customer_arrivals(env, theater))
# Execute
env.run(until=SIM_TIME)
print('... Done \n')

# Analysis/results
for movie in movies:
    num_renegers = theater.num_renegers[movie]
    lost_tickets = theater.lost_tickets[movie]
    num_renegers_impatience = theater.num_renegers_impatience[movie]
    lost_tickets_impatience = theater.lost_tickets_impatience[movie]

    print(f'Movie: "{movie}"')
    print(f'Number of people leaving due lack of tickets: {num_renegers}')
    print(f'Number of tickets lost due due lack of tickets: {lost_tickets}')
    print(f'Number of people leaving queue due impatience: {num_renegers_impatience}')
    print(f'Number of tickets lost due impatience: {lost_tickets_impatience}')
    print('-' * 30)        
    

Movie renege
Running Simulation...
... Done 

Movie: "Python Unchained"
Number of people leaving due lack of tickets: 53
Number of tickets lost due due lack of tickets: 159
Number of people leaving queue due impatience: 2
Number of tickets lost due impatience: 5
------------------------------
Movie: "Kill Process"
Number of people leaving due lack of tickets: 60
Number of tickets lost due due lack of tickets: 213
Number of people leaving queue due impatience: 7
Number of tickets lost due impatience: 21
------------------------------
Movie: "Pulp Implementation"
Number of people leaving due lack of tickets: 50
Number of tickets lost due due lack of tickets: 178
Number of people leaving queue due impatience: 5
Number of tickets lost due impatience: 20
------------------------------


Answer the following questions:

1. How many tickets are lost due lack of tickets and impatience for each movie?
1. How is this compared when there are no impatience *renegers*? 

**Answer in this markdown chunk:**

1. *Number of lost tickets:*
- Movie: "Python Unchained"
Number of tickets lost due due lack of tickets: 159 
Number of tickets lost due impatience: 5
- Movie: "Kill Process"
Number of tickets lost due due lack of tickets: 213
Number of tickets lost due impatience: 21
- Movie: "Pulp Implementation"
Number of tickets lost due due lack of tickets: 178
Number of tickets lost due impatience: 20

1. *The number of lost tickets decreases. Implementing impatience causes some customers to leave early, which reduces the number of tickets lost due to sellouts but introduces tickets lost due to impatience.*

### Task 4: What if?

Now we want to "optimize" our system in such a way that (hopefully) there are no tickets lost. For that we can only modify the number of tickets available (i.e. `TICKETS` parameter) which, due printing reasons, can only increase in multiples of 10 (50, 60, 70, etc.)

**Task 4: Find the minimum number of tickets available needed to ensure that no tickets are lost**.

**Note:** Assume the other parameters to be the same as before (i.e. `RANDOM_SEED = 42`, `SELLOUT_THRESHOLD = 2`, and `SIM_TIME = 120`)

In [80]:
#Change the TICKETS parameter to find the optimal number of tickets.

RANDOM_SEED = 42
TICKETS = 140  # Number of tickets per movie
SELLOUT_THRESHOLD = 2  # Fewer tickets than this is a sellout
SIM_TIME = 120  # Simulate until

# Setup and start the simulation
print('Movie renege')
print('Running Simulation...')
random.seed(RANDOM_SEED)
env = simpy.Environment()

#define resources
movies = ['Python Unchained', 'Kill Process', 'Pulp Implementation']
theater = Theater(env, movies, SELLOUT_THRESHOLD, TICKETS)

#define processes
env.process(customer_arrivals(env, theater))

# Execute
env.run(until=SIM_TIME)
print('... Done \n')
total = []
# Analysis/results
for movie in movies:
    num_renegers = theater.num_renegers[movie]
    lost_tickets = theater.lost_tickets[movie]
    num_renegers_impatience = theater.num_renegers_impatience[movie]
    lost_tickets_impatience = theater.lost_tickets_impatience[movie]

    print(f'Movie: "{movie}"')
    print(f'Number of people leaving due lack of tickets: {num_renegers}')
    print(f'Number of tickets lost due due lack of tickets: {lost_tickets}')
    print(f'Number of people leaving queue due impatience: {num_renegers_impatience}')
    print(f'Number of tickets lost due impatience: {lost_tickets_impatience}')
    print('-' * 30)      
    total_tickets = lost_tickets + lost_tickets_impatience
    total.append(total_tickets)
    print(f'Total of tickets lost: {total_tickets}')
    
total_lost_tickets = sum(total)
print(f'Total lost tickets for all movies: {total_lost_tickets}')

 

Movie renege
Running Simulation...
... Done 

Movie: "Python Unchained"
Number of people leaving due lack of tickets: 0
Number of tickets lost due due lack of tickets: 0
Number of people leaving queue due impatience: 25
Number of tickets lost due impatience: 77
------------------------------
Total of tickets lost: 77
Movie: "Kill Process"
Number of people leaving due lack of tickets: 0
Number of tickets lost due due lack of tickets: 0
Number of people leaving queue due impatience: 35
Number of tickets lost due impatience: 129
------------------------------
Total of tickets lost: 129
Movie: "Pulp Implementation"
Number of people leaving due lack of tickets: 0
Number of tickets lost due due lack of tickets: 0
Number of people leaving queue due impatience: 28
Number of tickets lost due impatience: 105
------------------------------
Total of tickets lost: 105
Total lost tickets for all movies: 311


In [83]:
num_tickets = None
min_total_lost_tickets = float('inf')

for i in range(13, 16):
    n = i * 10

    print('Running Simulation for', n, 'tickets...')
    random.seed(RANDOM_SEED)
    env = simpy.Environment()

    movies = ['Python Unchained', 'Kill Process', 'Pulp Implementation']
    theater = Theater(env, movies, SELLOUT_THRESHOLD, n)

    env.process(customer_arrivals(env, theater))

    env.run(until=SIM_TIME)
    print('... Simulation Done')

    total_lost_tickets_lack = sum(theater.lost_tickets[movie] for movie in movies)
    total_lost_tickets_impatience = sum(theater.lost_tickets_impatience[movie] for movie in movies)

    print('Total lost tickets because of lack for all movies with ', n, 'tickets:', total_lost_tickets_lack, '\n')
    print('Total lost tickets because of impatience for all movies with ', n, 'tickets:', total_lost_tickets_impatience, '\n')



Running Simulation for 130 tickets...
... Simulation Done
Total lost tickets because of lack for all movies with  130 tickets: 24 

Total lost tickets because of impatience for all movies with  130 tickets: 294 

Running Simulation for 140 tickets...
... Simulation Done
Total lost tickets because of lack for all movies with  140 tickets: 0 

Total lost tickets because of impatience for all movies with  140 tickets: 311 

Running Simulation for 150 tickets...
... Simulation Done
Total lost tickets because of lack for all movies with  150 tickets: 0 

Total lost tickets because of impatience for all movies with  150 tickets: 311 



In [58]:
num_tickets = None
min_total_lost_tickets = float('inf')

for i in range(1, 4):
    n = i * 10

    print('Running Simulation for', n, 'tickets...')
    random.seed(RANDOM_SEED)
    env = simpy.Environment()

    movies = ['Python Unchained', 'Kill Process', 'Pulp Implementation']
    theater = Theater(env, movies, SELLOUT_THRESHOLD, n)

    env.process(customer_arrivals(env, theater))

    env.run(until=SIM_TIME)
    print('... Simulation Done')

    total_lost_tickets_lack = sum(theater.lost_tickets[movie] for movie in movies)
    total_lost_tickets_impatience = sum(theater.lost_tickets_impatience[movie] for movie in movies)

    print('Total lost tickets because of lack for all movies with ', n, 'tickets:', total_lost_tickets_lack, '\n')
    print('Total lost tickets because of impatience for all movies with ', n, 'tickets:', total_lost_tickets_impatience, '\n')


Running Simulation for 10 tickets...
... Simulation Done
Total lost tickets because of lack for all movies with  10 tickets: 713 

Total lost tickets because of impatience for all movies with  10 tickets: 0 

Running Simulation for 20 tickets...
... Simulation Done
Total lost tickets because of lack for all movies with  20 tickets: 684 

Total lost tickets because of impatience for all movies with  20 tickets: 0 

Running Simulation for 30 tickets...
... Simulation Done
Total lost tickets because of lack for all movies with  30 tickets: 639 

Total lost tickets because of impatience for all movies with  30 tickets: 14 



Answer the following questions:

1. Is it possible to reduce the number of *renengers* (and tickets sold) due lack of tickets to zero (i.e. `lost_tickets = 0`)?, if so which is the minimum number of tickets available (i.e. `TICKETS` parameter) needed to ensure this? If no, why?

1. Is it possible to reduce the number of *renengers* (and tickets sold) due impatience to zero (i.e. `lost_tickets_impatience = 0`)?, if so which is the minimum number of tickets available (i.e. `TICKETS` parameter) needed to ensure this? If no, why?

**Answer in this markdown chunk:**

1. *To reduce the number of renegers due lack of tickets to zero you need 140 tickets available. It is the optimal number of tickets the cinema should release if the simulation time is 120, beacause we can see that increasing the amount of tickets after 140 doesn't change the number of lost tickets.*

1. *To have zero of renengers due impatience, the number of tickets should be 20. But it doesn't make sense because you loose 684 tickets due the lack of amount of tickets.*

### Task 5: Going beyond! (Optional)

This part is optional and just for fun =) 

**It won't be taken into account in the grading process of this assingment** and its sole purpose is to challenge yourself and have fun. 

Don't worry if you can't complete it, we will cover this topics in the upcoming lectures/exercises. Try your best!

#### 5.1. Optional Task 1

When chosing among alternatives (like movies), a common use case is to have different preferences for the alternatives. We model this with a probability distribution among alternatives.

Let's assume that our movies have different preferences. In particular:

- Python Unchained: 50%
- Kill Process: 30%
- Pulp Implementation: 20%

**Optional Task 1: Implement movie preferences**

**Hint:** The `random.choice` function only supports a uniform distribution over alternatives. However, we can use the `np.random.choice` function to support non-uniform distributions (through the `p` parameter). Check the following [reference](https://numpy.org/doc/stable/reference/random/generated/numpy.random.choice.html) for some examples.

**Note:** we have already imported numpy for you (`import numpy as np`) and we've also set the seed with `np.random.seed(RANDOM_SEED)`.

In [40]:
import numpy as np

In [41]:
class Customer:

    def __init__(self, env, theater):
        '''
        Initialize a customer entity.

        - env: the simulation environment
        - theater: the theater object
        '''

        self.env = env
        self.movie = random.choice(theater.movies)
        self.num_tickets = random.randint(1, 6)

        self.env.process(self.run(theater))
        
    def discuss(self):
        '''
        Customer discusses with the cashier about the movie being sold out.
        '''

        yield self.env.timeout(0.5)

    def run(self, theater):  
        '''
        Entity process for a customer buying tickets to a movie (flow)
        
        - theater: the theater object
        '''

        with theater.counter.request() as req:
            
            # Wait until it's our turn or until the movie is sold out
            result = yield req | theater.sold_out[self.movie]

            # Check if it's our turn or if movie is sold out
            if req not in result:
                theater.num_renegers[self.movie] += 1
                return

            # Check if enough tickets left.
            if theater.available[self.movie] < self.num_tickets:
                # Customer leaves after some discussion
                yield self.env.process(self.discuss())
                return

            yield self.env.process(theater.sell_tickets(self.movie, self.num_tickets))

In [42]:
RANDOM_SEED = 42
TICKETS = 50  # Number of tickets per movie
SELLOUT_THRESHOLD = 2  # Fewer tickets than this is a sellout
SIM_TIME = 120  # Simulate until

# Setup and start the simulation
print('Movie renege')
print('Running Simulation...')
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
env = simpy.Environment()

#define resources
movies = ['Python Unchained', 'Kill Process', 'Pulp Implementation']
theater = Theater(env, movies, SELLOUT_THRESHOLD, TICKETS)

#define processes
env.process(customer_arrivals(env, theater))

# Execute
env.run(until=SIM_TIME)
print('... Done \n')

# Analysis/results
for movie in movies:
    num_renegers = theater.num_renegers[movie]
    lost_tickets = theater.lost_tickets[movie]
    num_renegers_impatience = theater.num_renegers_impatience[movie]
    lost_tickets_impatience = theater.lost_tickets_impatience[movie]

    print(f'Movie: "{movie}"')
    print(f'Number of people leaving due lack of tickets: {num_renegers}')
    print(f'Number of tickets lost due due lack of tickets: {lost_tickets}')
    print(f'Number of people leaving queue due impatience: {num_renegers_impatience}')
    print(f'Number of tickets lost due impatience: {lost_tickets_impatience}')
    print('-' * 30)        

Movie renege
Running Simulation...
... Done 

Movie: "Python Unchained"
Number of people leaving due lack of tickets: 52
Number of tickets lost due due lack of tickets: 0
Number of people leaving queue due impatience: 0
Number of tickets lost due impatience: 0
------------------------------
Movie: "Kill Process"
Number of people leaving due lack of tickets: 39
Number of tickets lost due due lack of tickets: 0
Number of people leaving queue due impatience: 0
Number of tickets lost due impatience: 0
------------------------------
Movie: "Pulp Implementation"
Number of people leaving due lack of tickets: 53
Number of tickets lost due due lack of tickets: 0
Number of people leaving queue due impatience: 0
Number of tickets lost due impatience: 0
------------------------------


#### 5.2. Optional Task 2

With this new preferences, what if we could determine availability of tickets individually for each movie? How would the results change with those found in Task 4?

**Optional Task 2: Implement different ticket availability for each movie and find the minimum number of tickets (for each movie) needed to ensure that no tickets are lost. Compare your results with those found in Task 4.**

**Hint:** Use a dictionary to model the ticket availability for each movie. Something like: `TICKETS = {'Python Unchained': 50, 'Kill Process': 50, 'Pulp Implementation': 50}`

**Note:** Assume again that the number of tickets can only increase in multiples of 10.

In [43]:
class Theater:

    def __init__(self, env, movies, sellout_threshold, tickets):
        '''
        Initialize the theater.

        - env: the simulation environment
        - movies: a list of movie names
        - sellout_threshold: the number of tickets remaining at which a movie is considered sold out
        - tickets: the number of tickets available for each movie
        '''
        
        self.env = env
        self.counter = simpy.Resource(self.env, capacity=1)
        
        self.movies = movies
        self.sellout_threshold = sellout_threshold
        
        #"logging" attributes
        self.available = {movie: tickets for movie in self.movies}
        self.sold_out = {movie: self.env.event() for movie in self.movies}
        self.when_sold_out = {movie: None for movie in self.movies}
        self.num_renegers = {movie: 0 for movie in self.movies}
        
    def sell_tickets(self, movie, num_tickets):
        '''
        Process for selling tickets to a movie.

        - movie: the name of the movie
        - num_tickets: the number of tickets to sell
        '''

        # Buy tickets
        self.available[movie] -= num_tickets
        if self.available[movie] < self.sellout_threshold:
            # Trigger the "sold out" event for the movie
            self.sold_out[movie].succeed()
            self.when_sold_out[movie] = self.env.now
            self.available[movie] = 0
        
        yield self.env.timeout(1)

In [44]:
RANDOM_SEED = 42
TICKETS = 50  # Number of tickets per movie
SELLOUT_THRESHOLD = 2  # Fewer tickets than this is a sellout
SIM_TIME = 120  # Simulate until

# Setup and start the simulation
print('Movie renege')
print('Running Simulation...')
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
env = simpy.Environment()

#define resources
movies = ['Python Unchained', 'Kill Process', 'Pulp Implementation']
theater = Theater(env, movies, SELLOUT_THRESHOLD, TICKETS)

#define processes
env.process(customer_arrivals(env, theater))

# Execute
env.run(until=SIM_TIME)
print('... Done \n')

# Analysis/results
for movie in movies:
    num_renegers = theater.num_renegers[movie]
    lost_tickets = theater.lost_tickets[movie]
    num_renegers_impatience = theater.num_renegers_impatience[movie]
    lost_tickets_impatience = theater.lost_tickets_impatience[movie]

    print(f'Movie: "{movie}"')
    print(f'Number of people leaving due lack of tickets: {num_renegers}')
    print(f'Number of tickets lost due due lack of tickets: {lost_tickets}')
    print(f'Number of people leaving queue due impatience: {num_renegers_impatience}')
    print(f'Number of tickets lost due impatience: {lost_tickets_impatience}')
    print('-' * 30)              

Movie renege
Running Simulation...
... Done 



AttributeError: 'Theater' object has no attribute 'lost_tickets'

Answer the following questions:

- What can you say about this change? How is this compared with the results found in Task 4? 

**Answer in this markdown chunk:**

- *Your answer here*